In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults =  "../scripts/results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

#countryRoad = r'suc2cExudTip/none_1476_{}_10to14_20mn_0s_10/'
#countryRoad =  r'getsrivssriset/{}_1476_{}_10to25_20mn_0s_64/'
#countryRoad =  r'testspeed/thr4{}_1476_{}_10to25_20mn_0s_32/'
countryRoad = r'pdef/{}{}_1440_10to25_20mn_0s_32/'

In [4]:
# 1 vall per cell
nKonz = 8
volIdx=nKonz+1
coordId =volIdx+1 
# 1 vall per seg
lenIdx=coordId+1#volIdx+1
stIdx=lenIdx+1
orgIdId=stIdx+1
relLenId=orgIdId+1#segIdxid+1
segIdxid =relLenId+1

nToGet =segIdxid#elements + 3d coordinates of y , node+vol+len+st 
valnames=  ['wat','P','cl','coa',
            'cod','cca','ccd','css','co2',
            'vol','1dcoord',  '3dx', '3dy','3dz']
scenarios = ["baseline"]
setIds = [0,1,2,3]
exceptPset = []#[('baseline','19'),('baseline','47'),('baseline','83')]
#result_list_compExcept = [path2file.format(scenario, str(setId)) for scenario, setId in exceptPsets]
result_list_comp = [(str(setId),scenario) for scenario in scenarios for setId in setIds if (scenario, str(setId)) not in exceptPset ]

numPset = len(result_list_comp)
allPSets = [ i for i in range(numPset) if i not in exceptPset]

getAll = True
    

def getVal(mypathfilename,dtype=float,extension='.txt', header="infer",names = None):
    fullpath = pathresults +mypathfilename
    data_file_delimiter = ","
    if names is None:
        largest_column_count = 0
        with open(fullpath, 'r') as temp_f:
            lines = temp_f.readlines()
        last_line = lines[-1].strip()
        #print('last_line',last_line)
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        largest_column_count = len(array_from_last_line)
        names = [i for i in range(0, largest_column_count)]
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, 
                          header = header, names = names)
        #print('cst',cst.shape)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names)
    return cst


def get_last_line_as_numpy_array(file_path, dtype = float, verbose = False):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line

        try:
            last_line = lines[-1].strip()
        except:
            print('timeLine',file_path, len(lines))
            raise Exception

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        if verbose:
            print(file_path, len(array_from_last_line), end=", ")

        return array_from_last_line.copy()
    

def wrapCellData(df, valname,time,gId, lens, sts,gIds, scenario, pipi, nodes_X, nodes_Y, nodes_Z):
    df = pd.DataFrame(df)
    lId = np.where(gId == gIds)
    df['time']=np.array(list(time[-df.shape[0]:]))
    df['st']=sts[gId]
    df['lens']=np.array(list(lens.loc[:,gId].copy()[-df.shape[0]:]))# to list to reset index
    df['scenario']=scenario
    df['gId'] = gId
    df['pSet']=pipi
    df['vals']=valname
    df['3dx']=np.array(list( nodes_X.loc[:, gId+1].copy()[-df.shape[0]:]))
    df['3dy']= np.array(list(nodes_Y.loc[:, gId+1].copy()[-df.shape[0]:]))
    df['3dz']=np.array(list( nodes_Z.loc[:, gId+1].copy()[-df.shape[0]:]))
    try:
        assert not np.isnan(df['lens'].values).any()
    except:
        print('wrapCellData error')
        print(df)
        raise Exception
    return df

def getCellData(cid,gId,path2file, verbose = False):
    if verbose :
        print('getData_',cid,gId,path2file, volIdx,coordId)
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = getVal(path2file+fileName, dtype = float)
    if cid == volIdx:
        return cVol
    elif cid <= nKonz:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = getVal(path2file+fileName,
                                                 dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return cVol
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = getVal(path2file+fileName, dtype = float)
        konz = Q1/cVol #mol/cm3 or cm3/cm3
        return konz
    elif cid == coordId:
        fileName = "cyl_val/Cyl_coord_"+str(gId)+".txt"
        Q1 = getVal(path2file+fileName,
                                          dtype = float)

        return Q1
    raise Exception
    
GinitspdS = []
for pSet in result_list_comp:
    print(pSet[0],pSet[1])
    path2file = countryRoad.format(pSet[0],pSet[1])

    time = np.array(list(
        pd.read_csv(pathresults + path2file + "time.txt", 
                       names = ["time","Qlight"])["time"][1:])) # because we have twice the initial value

    rr = get_last_line_as_numpy_array(path2file+"rhizoSegsId.txt", dtype = int)
    lens = getVal(path2file+"length_Segs"+".txt", dtype =float)
    dx3 = getVal(path2file+"nodes_X"+".txt", dtype =float)
    dy3 = getVal(path2file+"nodes_Y"+".txt", dtype =float)
    dz3 = getVal(path2file+"nodes_Z"+".txt", dtype =float)
    sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float, 
                                   verbose = False)# last time steps 
    Ginits = [pd.concat([wrapCellData(df=getCellData( cid, int( gId_),path2file), #a(df, time,gId, lens, sts)
                                 valname=valnames[cid],time=time,gId=int(gId_), lens=lens, sts=sts,
                        gIds=  rr, nodes_X =dx3, nodes_Y=dy3, nodes_Z=dz3,
                                scenario=pSet[0],pipi=pSet[1]) for gId_ in rr] , ignore_index=True)
                   for cid in range(coordId+1)]
    Ginitspd = pd.concat(Ginits, ignore_index=True)
    GinitspdS.append(Ginitspd)


0 baseline


FileNotFoundError: [Errno 2] No such file or directory: '../scripts/results/pdef/0baseline_1440_10to25_20mn_0s_32/rhizoSegsId.txt'

In [3]:
GinitspdSdf = pd.concat(GinitspdS, ignore_index=True)

In [4]:
GinitspdSdf.reset_index(drop=True, inplace=True)

In [5]:
GinitspdSdf.to_pickle('./pset3_allcell_A.pkl')

In [6]:
GinitspdSdf

,0,1,2,3,4,5,6,7,8,time,st,lens,scenario,gId,pSet,vals,3dx,3dy,3dz
0,0.000791,0.001166,0.001719,0.002534,0.003735,0.005505,0.008114,0.011960,0.017628,10.013889,0.0,1.0,earlyDry,0,5,wat,-0.298434,-0.304855,-1.204434
1,0.000791,0.001166,0.001719,0.002534,0.003735,0.005505,0.008114,0.011960,0.017628,10.027778,0.0,1.0,earlyDry,0,5,wat,-0.298434,-0.304855,-1.204434
2,0.000791,0.001166,0.001719,0.002534,0.003735,0.005505,0.008114,0.011960,0.017628,10.041667,0.0,1.0,earlyDry,0,5,wat,-0.298434,-0.304855,-1.204434
3,0.000791,0.001166,0.001719,0.002534,0.003735,0.005505,0.008114,0.011960,0.017628,10.055556,0.0,1.0,earlyDry,0,5,wat,-0.298434,-0.304855,-1.204434
4,0.000791,0.001166,0.001719,0.002534,0.003735,0.005505,0.008114,0.011960,0.017628,10.069444,0.0,1.0,earlyDry,0,5,wat,-0.298434,-0.304855,-1.204434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32660073,0.024494,0.035501,0.051455,0.074579,0.108093,0.156670,0.227075,0.329121,0.477024,24.944444,1.0,1.0,lateDry,417,61,1dcoord,-0.610360,1.138855,-7.048179
32660074,0.024494,0.035501,0.051455,0.074579,0.108093,0.156670,0.227075,0.329121,0.477024,24.958333,1.0,1.0,lateDry,417,61,1dcoord,-0.610360,1.138855,-7.048179
32660075,0.024494,0.035501,0.051455,0.074579,0.108093,0.156670,0.227075,0.329121,0.477024,24.972222,1.0,1.0,lateDry,417,61,1dcoord,-0.610360,1.138855,-7.048179
32660076,0.024494,0.035501,0.051455,0.074579,0.108093,0.156670,0.227075,0.329121,0.477024,24.986111,1.0,1.0,lateDry,417,61,1dcoord,-0.610360,1.138855,-7.048179
